<a href="https://colab.research.google.com/github/thmshreyas/Detect-Emotions-of-your-Favourite-Cartoons/blob/main/Detect_Emotions_of_your_Favourite_Cartoons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<h2 style="color:#4A90E2;">Detect Emotions of Your Favourite Cartoons</h2>
<p><b>Samyak Sharma</b> - 1DS22CS190<br>
<b>Shashwat Kumar</b> - 1DS22CS199<br>
<b>Shaurya Katiyar</b> - 1DS22CS200<br>
<b>Shivansh Karan</b> - 1DS22CS202<br>
<b>Shreyas T H M</b> - 1DS22CS208</p>
<hr>
<ul>
  

     
     

In [ ]:
#Importing necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import math
%matplotlib inline

In [ ]:
#This section (commented out) would extract frames from a training video at regular intervals
# videoFile = "Dataset/Train Tom and Jerry.mp4"
# imageFolder = "Dataset/frameTrain"
# cap = cv2.VideoCapture(videoFile)
# frameRate = cap.get(5)
# count = 0
# while( cap.isOpened() ):
#     frameId = cap.get(1)
#     ret, frame = cap.read()
#     if(ret != True):
#         break
#     if(frameId % math.floor(frameRate) == 0 ):
#         filename = imageFolder + "/frame" + str(count) + ".jpg"
#         cv2.imwrite(filename,frame)
#         count = count + 1

# cap.release()
# print("Done!")

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/datasciencesatasets/archive/96714c94-6-Dataset/Dataset/Train.csv')
test  = pd.read_csv('/content/drive/MyDrive/datasciencesatasets/archive/96714c94-6-Dataset/Dataset/Test.csv')

In [ ]:
train.shape
s = train['Emotion'].tolist()  # Converts emotion column to list
train['Emotion'].unique()

array(['happy', 'surprised', 'angry', 'Unknown', 'sad'], dtype=object)

In [ ]:
#Label Encoding
tg_dict = {"happy":0, "surprised":1, "angry":2, "Unknown":3, "sad":4}
def label_encode(x):
    return tg_dict[x]

train['Emotion'] = train['Emotion'].apply(label_encode)
frame_ID = train['Frame_ID'].tolist()
emotion = train['Emotion'].tolist()

In [ ]:
from tensorflow import keras
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
from tensorflow.keras.layers import Dense, Activation, Flatten,Input
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import *
import os
import glob
import tensorflow as tf

In [ ]:
features=[]
labels=[]
path = '/content/drive/MyDrive/datasciencesatasets/archive/frames/train_frames/' # Added / at the end of the path
for i in range(0,298):
  if os.path.isfile(path+str(frame_ID[i])): # Added .jpg extension to the file name
    pic = image.load_img(path+str(frame_ID[i]), target_size=(224, 224))
    #print(path+str(images[i]))
    x = image.img_to_array(pic)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features.append(x)
    labels.append(emotion[i])
  else:
    print(path+str(frame_ID[i]), 'not present')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
len(features)

298

In [ ]:
npfeatures = np.array(features)
print(npfeatures.shape)
img_dt = np.rollaxis(npfeatures, 1, 0)
print(img_dt.shape)
X = img_dt[0]
print(X.shape)
labels = np.array(emotion)
Y = to_categorical(emotion,5)
print(Y.shape)
# print(Y)


(298, 1, 224, 224, 3)
(1, 298, 224, 224, 3)
(298, 224, 224, 3)
(298, 5)


In [ ]:
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
IMAGE_SIZE=[224,224]
pretrained_model = MobileNet(weights='imagenet', include_top=False ,input_shape=[*IMAGE_SIZE, 3])
pretrained_model.trainable = False

17225924/17225924 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [ ]:
model = Sequential([
        pretrained_model,
        GlobalAveragePooling2D(),
        Dense(60, activation='relu'),
        Dense(30, activation='relu'),

        Dense(5, activation='softmax')
    ])
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=4, verbose=0, mode='auto', baseline=None, restore_best_weights=False)




In [ ]:
model.compile(optimizer = 'adamax', loss='categorical_crossentropy', metrics=[f1_m])
model.fit(X, Y, batch_size=32, epochs=10, validation_split=.1,callbacks=[es])

Epoch 1/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 16s 855ms/step - f1_m: 0.1166 - loss: 1.5792 - val_f1_m: 0.8000 - val_loss: 0.8786
Epoch 2/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - f1_m: 0.3765 - loss: 1.2387 - val_f1_m: 0.7234 - val_loss: 0.6670
Epoch 3/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - f1_m: 0.4656 - loss: 1.1494 - val_f1_m: 0.8235 - val_loss: 0.5750
Epoch 4/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - f1_m: 0.5794 - loss: 0.9194 - val_f1_m: 0.8000 - val_loss: 0.6000
Epoch 5/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - f1_m: 0.5319 - loss: 0.9633 - val_f1_m: 0.8235 - val_loss: 0.5999
Epoch 6/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - f1_m: 0.6001 - loss: 0.8369 - val_f1_m: 0.8235 - val_loss: 0.5431
Epoch 7/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - f1_m: 0.6685 - loss: 0.7652 - val_f1_m: 0.8077 - val_loss: 0.5361
Epoch 8/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - f1_m: 0.6882 - loss: 0.7144 - val_f1_m: 0.8000 - val_loss: 0.6153
Epoch 9/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - f1_m: 0.712

In [ ]:
images_test = test['Frame_ID'].tolist()
test_features=[]
path_test = '/content/drive/MyDrive/datasciencesatasets/archive/frames/test_frames/'

In [ ]:
len(images_test)


186

In [ ]:
for i in range(0,186):
  if os.path.isfile(path_test+str(images_test[i])):
    pic = image.load_img(path_test+str(images_test[i]), target_size=(224, 224))
    #print(path+str(images[i]))
    x = image.img_to_array(pic)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    test_features.append(x)
  else:
    print(path_test+str(images_test[i]), 'not present')

In [ ]:
test_features = np.array(test_features)
print(test_features.shape)
test_features = np.rollaxis(test_features, 1, 0)
print(test_features.shape)
X_test = test_features[0]
print(X_test.shape)

(186, 1, 224, 224, 3)
(1, 186, 224, 224, 3)
(186, 224, 224, 3)


In [ ]:
preds = model.predict(X_test)

6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 679ms/step


In [ ]:
predictions=[]
for i in preds:
    predictions.append(np.argmax(i))

In [ ]:
len(predictions)

186

In [ ]:
test['Emotion'] = predictions

In [ ]:
gt_dict = dict((v,k) for k,v in tg_dict.items())

def inverse_encode(x):
    return gt_dict[x]

test['Emotion'] = test['Emotion'].apply(inverse_encode)

In [ ]:
test.head(1)

,Frame_ID,Emotion
0,test0.jpg,angry


In [ ]:
test.to_csv('Submission.csv',header=True,index = None)